In [2]:
import requests
import pandas as pd


 
access_token = get_access_token()


# Specify the fields you want to retrieve
fields = 'text,create_time'

# Specify the video IDs for which you want to retrieve comments
video_ids = ['7329921177290394913','7340996027131923744','7351029721129438496','7363617544655162657','7327662103689497889','7340579138538933537','7350658359378349345',
             '7361771387968556321','7326889646439320865','7338837326954646817','7350211359562272033','7359586826618801440','7325462089521204513','7338831583924800801',
             '7349193034799615265','7359188457329151265','7324695615470898465','7338324079692320032','7348104686903332128','7358763488019303712','7324352628085804321',
             '7334658038780955936','7345940717707005217','7358444423086116129','7324235869420834080','7333217469483470112','7345841218695318816','7356973659862486304',
             '7322070989825756448','7343178083362491681','7356859297906658592','7320554412395531552','7342450397107309857','7356186873217174816','7341343651336949024',
             '7355825335402794273'] # Add more video IDs as needed

# List to store all comments
all_data = []

# Iterate over each video ID
for video_id in video_ids:
    # Make the request to retrieve comments for the current video ID
    isFetchingComments = True
    cursor = 0
    MAX_COUNT = 100
    while isFetchingComments:
        response = requests.post(
            'https://open.tiktokapis.com/v2/research/video/comment/list/?fields=%s' % fields,
            headers={'authorization': 'bearer ' + access_token},
            data={ 
                'video_id': video_id,
                'max_count': MAX_COUNT,
                'cursor': cursor
            }
        )
        
        # Extract comments from the response
        data = response.json()
        comments = data.get('data', {}).get('comments', [])
        
        # Append comments to the list
        for comment in comments:
            comment['video_id'] = video_id
            all_data.append(comment)
        
        if len(comments) == 0:
            isFetchingComments = False
            
        else:
            cursor += MAX_COUNT

# Create DataFrame
df = pd.DataFrame(all_data)

print(df)

      create_time                                               text  \
0      1706630863                                     Ampel beenden.   
1      1707369852                              Der ist ja bald weg 😂   
2      1706633846  Gestalten durch nicht investieren. Ach Mann de...   
3      1706758539  ach Gott. die opportunisitsche Umfallertruppe ...   
4      1707419643                                       Luftpumpe😂😂😂   
...           ...                                                ...   
2672   1712849009  Aber das ist doch kein Durchschnitt, dass ist ...   
2673   1712848617  Glaubst du einer denkt:“Heute ist ein schöner ...   
2674   1712844426  Impfung hat vor schweren Krankheitsverläufen g...   
2675   1713071783  Es zeigt nur deutlich, wir brauchen die CDU wi...   
2676   1713089763                Kommt bisschen spät das Video oder?   

                 video_id  
0     7329921177290394913  
1     7329921177290394913  
2     7329921177290394913  
3     73299211772903949

In [3]:
#### CONVERT DATE SO IT IS LATER READABLE IN R:

import pandas as pd

# Step 1: Create a sample DataFrame

df = pd.DataFrame(all_data)

# Step 2: Convert the Unix timestamps to human-readable date and time
df['create_time_readable'] = pd.to_datetime(df['create_time'], unit='s')

# Display the DataFrame
print(df)

      create_time                                               text  \
0      1706630863                                     Ampel beenden.   
1      1707369852                              Der ist ja bald weg 😂   
2      1706633846  Gestalten durch nicht investieren. Ach Mann de...   
3      1706758539  ach Gott. die opportunisitsche Umfallertruppe ...   
4      1707419643                                       Luftpumpe😂😂😂   
...           ...                                                ...   
2672   1712849009  Aber das ist doch kein Durchschnitt, dass ist ...   
2673   1712848617  Glaubst du einer denkt:“Heute ist ein schöner ...   
2674   1712844426  Impfung hat vor schweren Krankheitsverläufen g...   
2675   1713071783  Es zeigt nur deutlich, wir brauchen die CDU wi...   
2676   1713089763                Kommt bisschen spät das Video oder?   

                 video_id create_time_readable  
0     7329921177290394913  2024-01-30 16:07:43  
1     7329921177290394913  2024-02-08

In [10]:
#Check working directory
import os

# Check current working directory
print(os.getcwd())

# Save to the current working directory
df.to_csv('/Users/emma/Desktop/FDP_comments.csv', index=False)

/


In [22]:
# This is our classification prompt
PROMPT =  "Conduct a sentiment analysis of the comments on TikTok Vidoes by the official social media profiles of German parliamentarian parties on TikTok. The comments are generated in German. Consider the tone, context, and language and emojis used in each comment to determine its sentiment.  Respond with '0' if the comment contains neutral sentiment, respond with '1' if the content contains positive sentiment, and '2' if the comment contains negative sentiments. No explanation is needed. Here are some examples to guide your analysis. The comment 'Das nenn ich mal eine gute Partei',  the sentiment should be categorized as positive, so 1. If the comment is 'Ich habe keine starken Gefühle zu diesem Thema', the sentiment should be catgeorized as neutral, so 0. An example for a negative sentiment comment is 'Wenn das böse ein Gesicht hat', so it should be categorizes as 2." 

In [23]:
import openai
from openai import OpenAI

In [24]:
#We define which model to use throughout
MODEL = 'gpt-3.5-turbo-0125' # We use the cheap model 
WAIT_TIME = 0.01 # Wait time between each request. This depends on the rate limit of the model used

client = OpenAI(
    api_key= "" #Set the API key. See the how-to guide for further instructions
)

In [25]:
# Simple function for analyzing a message with OpenAI
def analyze_message(text, temperature=0.2):
    try:
        response = client.chat.completions.create(
            model = MODEL, 
            temperature=temperature,
            messages=[
                    {"role": "system", "content": f"'{PROMPT}'"}, #The system instruction tells the bot how it is supposed to behave
                    {"role": "user", "content": f"'{text}'"} #This provides the text to be analyzed.
                ]
        )
        result = ''
        for choice in response.choices:
            result += choice.message.content
        print(f"{text}: {result}")
        return result 
    except Exception as e:
        # Not the prettiest solution; but easy
        print(f'Exception happened: {e}')
        return 'error'

In [26]:
analyze_message("'Euch würde ich nicht wählen!'")

'Euch würde ich nicht wählen!': 2


'2'

In [27]:
import pandas as pd

# Specify the path to the CSV file
file_path = '/Users/emma/Desktop/FDP_cleaned_data.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
print(df)

      create_time                                               text  \
0      1706630863                                     Ampel beenden.   
1      1707369852                              Der ist ja bald weg 😂   
2      1706633846  Gestalten durch nicht investieren. Ach Mann de...   
3      1706758539  ach Gott. die opportunisitsche Umfallertruppe ...   
4      1707419643                                       Luftpumpe😂😂😂   
...           ...                                                ...   
2456   1712849009  Aber das ist doch kein Durchschnitt, dass ist ...   
2457   1712848617  Glaubst du einer denkt:“Heute ist ein schöner ...   
2458   1712844426  Impfung hat vor schweren Krankheitsverläufen g...   
2459   1713071783  Es zeigt nur deutlich, wir brauchen die CDU wi...   
2460   1713089763                Kommt bisschen spät das Video oder?   

                 video_id create_time_readable  
0     7329921177290394624  2024-01-30 16:07:43  
1     7329921177290394624  2024-02-08

In [28]:
# We start by defining a new column with the result
df['sentiment'] = None

# Save the DataFrame to a CSV file
df.to_csv('/Users/emma/SEN_FDP.csv', compression='gzip', index=False)

In [29]:
import time 
i = 0
while(True):

    #Find all unprocessed lines; where 
    left = df.loc[df['sentiment'].isna()]
    
    #No lines left? Then we're done
    if len(left)==0:
        print("All done!")
        break
        
    #Take a random line
    line = left.sample()
    index = line.index.values[0]
    text = line['text'].values[0]
    
    print(f"There are {len(left)} left to process. Processing: {index}")
    
    #Wait for a bit, to not overload the API
    time.sleep(WAIT_TIME)
    
    #Analyze the specific line, chunk by chunk    
    result = analyze_message(text)
    df.loc[index,'sentiment'] = result 

    i+=1    
    #Save the result to persistent file
    df.to_csv('/Users/emma/Sentiment_FDP.csv.gz',compression='gzip',index=False)

There are 2461 left to process. Processing: 862
Wir sind die EU...jedes Land für sich gibt's nicht mehr auf lange Sicht ...der Norden muss vereint werden^^: 1
There are 2460 left to process. Processing: 1285
frau?: 0
There are 2459 left to process. Processing: 2199
Ja das stimmt Frau Strack Zimmerman Sie setzen sich wenigstens durch.Geben Sie nichts auf Börse Worte.Ihnen alles gute.😳: 1
There are 2458 left to process. Processing: 258
W FDP: 2
There are 2457 left to process. Processing: 2243
@ProfLeertast: 0
There are 2456 left to process. Processing: 2382
Buuuuuuh: 2
There are 2455 left to process. Processing: 337
Pseudoliberalen: 2
There are 2454 left to process. Processing: 1062
das ist ein böser Mann 😳😳: 2
There are 2453 left to process. Processing: 582
🤮🤮🤮🤮🤮🤮🤮🤮🤮🤮: 2
There are 2452 left to process. Processing: 418
@antonia lmao: 0
There are 2451 left to process. Processing: 1856
Im Ernst 🙈: 2
There are 2450 left to process. Processing: 626
@Vanessa: 0
There are 2449 left to process.

In [30]:
df.to_csv('/Users/emma/FDP_done.csv',index=False)

In [31]:
#Check if data is the way we want
print(df.head(10))

   create_time                                               text  \
0   1706630863                                     Ampel beenden.   
1   1707369852                              Der ist ja bald weg 😂   
2   1706633846  Gestalten durch nicht investieren. Ach Mann de...   
3   1706758539  ach Gott. die opportunisitsche Umfallertruppe ...   
4   1707419643                                       Luftpumpe😂😂😂   
5   1707447235  Alter Deutschland tut mir so leid. Aber Österr...   
6   1707952575  bitte??? die grünen Vollidioten versteuern doc...   
7   1706739881            Baron von Schwurbelhausen in flames 🥳💥🙊   
8   1714284228            Ja unser Geld zum Fenster rausschmeisst   
9   1706848877  fdp klein halten👌 danke für Ihre Arbeit 👌 <5% ...   

              video_id create_time_readable sentiment  
0  7329921177290394624  2024-01-30 16:07:43         2  
1  7329921177290394624  2024-02-08 05:24:12         1  
2  7329921177290394624  2024-01-30 16:57:26         2  
3  7329921177290